<div style="text-align: right" align="right"><i>Peter Norvig, December 2024</i></div>

# Advent of Code 2024

I  enjoy doing the [**Advent of Code**](https://adventofcode.com/) (AoC) programming puzzles, so here we go for 2024!  Our old friend [@GaryJGrady](https://x.com/garyjgrady) is here to provide illustrations:

<a href="https://x.com/garyjgrady"><img src="https://pbs.twimg.com/media/Gdp709FW8AAq2_m?format=jpg&name=medium" width=400 alt="GaryJGrady cartoon"></a>

I traditionally start by loading up my [**AdventUtils.ipynb**](AdventUtils.ipynb) notebook (same as last time except for the `current_year`):

In [1]:
%run AdventUtils.ipynb
current_year = 2024

Each day's solution consists of three parts, making use of my `parse` and `answer` utilities:
- **Reading the day's input**. E.g. `pairs = parse(1, ints)`. 
- **Solving Part One**. Find the solution and record it with, e.g., `answer(1.1, 4, lambda: 2 + 2)`.
- **Solving Part Two**. Find the solution and record it with, e.g., `answer(1.2, 9, lambda: 3 * 3)`.

The function `parse` assumes that the input is a sequence of records (default one per line), each of which should be parsed in some way (default just left as a string, but the argument `ints` says to treat each record as a tuple of integers). The function `answer` records the correct answer (for regression testing), and records the run time (that's why a `lambda:` is used).

To fully understand each day's puzzle, and to follow along the drama involving Santa, the elves, the Chief Historian, and all the rest, you need to read the descriptions on the [**AoC**](https://adventofcode.com/) site, as linked in the header for each of my day's solutions, e.g. [**Day 1**](https://adventofcode.com/2023/day/1) below. Since you can't read Part 2 until you solve Part 1, I'll partially describe Part 2 in this notebook. But I can't copy the content of AoC here, nor show my input files; you need to go to the site for that.



 

# [Day 1](https://adventofcode.com/2024/day/1) Historian Hysteria

According to the narrative, North Pole Historians created two lists of **location IDs**. We can parse them as a sequence of pairs of integers, and then use the transpose function, `T` to get two lists of ID numbers:

In [2]:
left, right = location_ids = T(parse(1, ints))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1000 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
38665   13337
84587   21418
93374   50722
68298   57474
54771   18244
49242   83955
66490   44116
65908   51323
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 1000 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(38665, 13337)
(84587, 21418)
(93374, 50722)
(68298, 57474)
(54771, 18244)
(49242, 83955)
(66490, 44116)
(65908, 51323)
...


<img src="https://pbs.twimg.com/media/GdvPVOpXcAEZ34_?format=jpg&name=medium" width=400>

### Part 1: What is the total distance between your lists?

The **distance** between two numbers is the absolute value of their difference, and the **total distance** between two lists is the sum of the distances between respective pairs, where "respective" means to sort each list and then take the distance between the first element of each list, plus the distance between the second element of each list, and so on. (I use the transpose utility function, `T`, to turn the input sequence of 1000 pairs into two lists, each of 1000 integers.)

In [3]:
def total_distance(left: Ints, right: Ints) -> int:
    """Total distance between respective list elements, after sorting."""
    return sum(abs(a - b) for a, b in zip(sorted(left), sorted(right)))

answer(1.1, 1830467, lambda:
       total_distance(left, right))

Puzzle  1.1:   .0002 seconds, answer 1830467         ok

### Part 2: What is their similarity score?

The **similarity score** is the sum of each element of the left list times the number of times that value appears in the right list.

In [4]:
def similarity_score(left: Ints, right: Ints) -> int:
    """The sum of each x in `left` times the number of times x appears in `right`."""
    counts = Counter(right)
    return sum(x * counts[x] for x in left)

answer(1.2, 26674158, lambda:
       similarity_score(left, right))

Puzzle  1.2:   .0002 seconds, answer 26674158        ok

# [Day 2](https://adventofcode.com/2024/day/2): Red-Nosed Reports

Today's input is a sequence of **reports**, each of which is a sequence of integers:

In [5]:
reports = parse(2, ints)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1000 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
74 76 78 79 76
38 40 43 44 44
1 2 4 6 8 9 13
65 68 70 72 75 76 81
89 91 92 95 93 94
15 17 16 18 19 17
46 47 45 48 51 52 52
77 78 79 82 79 83
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 1000 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(74, 76, 78, 79, 76)
(38, 40, 43, 44, 44)
(1, 2, 4, 6, 8, 9, 13)
(65, 68, 70, 72, 75, 76, 81)
(89, 91, 92, 95, 93, 94)
(15, 17, 16, 18, 19, 17)
(46, 47, 45, 48, 51, 52, 52)
(77, 78, 79, 82, 79, 83)
...


### Part 1: How many reports are safe?

A report is **safe** if it is  monotonically strictly increasing or strictly decreasing, and if no difference between adjacent numbers is greater than 3 in absolute value.

In [6]:
def safe(report: Ints) -> bool:
    """A report is safe if all differences are either in {1, 2, 3} or in {-1, -2, -3}."""
    deltas = diffs(report)
    return deltas.issubset({1, 2, 3}) or deltas.issubset({-1, -2, -3})
    
def diffs(report: Ints) -> set:
    """The set of differences between adjacent numbers in the report."""
    return {report[i] - report[i - 1] for i in range(1, len(report))}

assert diffs((7, 6, 4, 2, 1)) == {-1, -2}
assert safe((7, 6, 4, 2, 1))  == True

In [7]:
answer(2.1, 257, lambda:
       quantify(reports, safe))

Puzzle  2.1:   .0011 seconds, answer 257             ok

### Part 2: How many reports are safe using the Problem Dampener?

The **problem dampener** says that a report is safe if you can drop one element and get a safe report.

In [8]:
def safe_with_dampener(report: Ints) -> bool:
    """Is there any way to drop one element of `report` to get a safe report?"""
    return any(map(safe, drop_one(report)))

def drop_one(report) -> Iterable:
    """All ways of dropping one element of the input report."""
    return (report[:i] + report[i + 1:] for i in range(len(report)))

assert set(drop_one('1234')) == {'234', '134', '124', '123'}

In [9]:
answer(2.2, 328, lambda:
       quantify(reports, safe_with_dampener))

Puzzle  2.2:   .0077 seconds, answer 328             ok

# [Day 3](https://adventofcode.com/2024/day/3): Mull It Over

Today's input is a computer program with some corrupted characters. The program has multiple lines, but lines don't matter, so I will concatenate them into one big string:

In [10]:
program = cat(parse(3))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 6 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
where(536,162)~'what()what()how(220,399){ mul(5,253);mul(757,101)$where()@why()who()&when()from( ...
}?~who()select()-mul(316,505)&%*how()mul(363,589)>?%-:)where()~{{mul(38,452)select()%>[{]%>%mul( ...
?>where(911,272)'mul(894,309)~+%@#}@#why()mul(330,296)what()mul(707,884)mul;&}<{>where()$why()]m ...
> (when()[where()/#!/usr/bin/perl,@;mul(794,217)select():'])select()mul(801,192)why()&]why()/:]* ...
,+who():mul(327,845)/ >@[>@}}mul(86,371)!~&&~how(79,334)mul(637,103)why()mul(358,845)-#~?why(243 ...
where()#{*,!?:$mul(204,279)what()!{ what()mul(117,94)!select()>:mul(665,432)#don't()!!<!? mul(50 ...


### Part 1: What do you get if you add up all of the results of the multiplications?

For Part 1, just look for instructions of the form "mul(*digits*,*digits*)", using a regular expression and `re.findall`. Perform each of these multiplications and add them up, and ignore all other characters/instructions:

In [11]:
def execute(program: str) -> int:
    """The sum of the results of the multiply instructions."""
    return sum(prod(ints(m)) for m in multiplications(program))

def multiplications(program: str) -> List[str]:
    """A list of all the multiplication instructions in the program."""
    return re.findall(r'mul\(\d+,\d+\)', program)

test = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
assert execute(test) == 161
assert multiplications(test) == ['mul(2,4)', 'mul(5,5)', 'mul(11,8)', 'mul(8,5)']

In [12]:
answer(3.1, 156388521, lambda: 
       execute(program))

Puzzle  3.1:   .0014 seconds, answer 156388521       ok

### Part 2: What do you get if you add up all of the results of just the enabled multiplications?

For Part 2, the instruction "don't()"  says to disable (ignore) following multiply instructions until a "do()" instruction enables them again. I will define the function `enabled`, which returns the part of the program that is enabled, by susbstituting a space for the "don't()...do()" sequence.

In [13]:
def enabled(program: str) -> str:
    """Just the part of the program that is enabled; remove "don't()...do()" text."""
    return re.sub(r"don't\(\).*?(do\(\)|$)", " ", program)

test2 = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
assert enabled(test2) == 'xmul(2,4)&mul[3,7]!^ ?mul(8,5))'
assert execute(enabled(test2)) ==  2 * 4 + 8 * 5 == 48
assert multiplications(enabled(test2)) == ['mul(2,4)', 'mul(8,5)']

In [14]:
answer(3.2, 75920122, lambda:
       execute(enabled(program)))

Puzzle  3.2:   .0009 seconds, answer 75920122        ok

# [Day 4](https://adventofcode.com/2024/day/4): Ceres Search

Today's puzzle is a 2D word-search puzzle:

In [15]:
xmas_grid = Grid(parse(4))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 140 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
MASAMXMSSXXMAMXXMXMASXMASXMMSMSMMMAXMASASMMSSMSXAXMASMMSMMMSSMSASMSSSSMSMSMXXMXMAXAMXMSMSSXSAMXM ...
MASMMXMASAXASMSMMMSAMXSMSAMXAAAAAXAMXASXAMAAAAMMSMMMMMASXAAAAMMAMAMMASAAAAXMXMSSSSSSMMSAMAXAXXSM ...
MMXAXMMMSXMAMAAXAAXAAAXXSMMSMSMSMXAXMXSMMMMSSMXAMXAAXMAMMMMSSMMAMAMMAMMMMMXSAAXAAMMAXXSAMXMSMAXM ...
SXSAMASASMSXMSMSMSSMMMMMMXAMXMMXMASMMMMAXXAAAMMMSSSSSMASXXAAXASMSXXMXSXSXSASMSMMSMSAMMMAMXAAMASX ...
AAAXXXMASASXMXMAXXMMASAASMXSASASXAAAAMSSMMMSXMAAMMMMMXAXMMMMSAMXAMASAMXSAMASXXAXAAMAMXSAMXSXSMMA ...
MSMMXXMMMAMAMMMMMMXSAXXAMMMMXSAXMMXXAMXAAMMXMASXMAAASMMXAAMXAXAMMMAMAMAMAMXMASXMMXMAAXMAXMAMXMSA ...
MXAXAMXXMMMMSAMAASMMMSMMASASAMAMAXMSXMSMMXAMXAXMMSSXSASXSSSMAMSMXMXSAMSSSMAMXMXAMAXXMMSAXAXMMXMA ...
ASXMMXSAMXAASXXMXSAAAXASAMMMASMSSSMAAMMXMMSSMASAMAMMMAMMAXMAXMASXM

### Part 1: How many times does XMAS appear?

We just have to find how many times the word "XMAS" appears in the grid, horizontally, vertically, or diagonally, forwards or backwards.  The variable `directions8` contains those eight directions (as (delta-x, delta-y) pairs). So examine each square of the grid and if it contains "X", see in how many of the directions it spells "XMAS". (Note that locations in the grid are denoted by `(x, y)` coordinates, as are directions (e.g., `(1, 0)` is the `East` direction. The functions `add` and `mul` do addition and scalar multiplication on these vectors.)

In [16]:
def word_search(grid: Grid, word='XMAS') -> int:
    """How many times does the given word appear in the grid?"""
    return quantify(grid_can_spell(grid, start, dir, word) 
                    for start in grid 
                    if grid[start] == word[0]
                    for dir in directions8)

def grid_can_spell(grid, start, dir, word):
    """Does `word` appear in grid starting at `start` and going in direction `dir`?"""
    return all(grid[add2(start, mul(dir, i))] == word[i] for i in range(len(word)))

In [17]:
answer(4.1, 2401, lambda:
       word_search(xmas_grid))

Puzzle  4.1:   .0742 seconds, answer 2401            ok

### Part 1: How many times does an X-MAS appear?

Upon further review, the goal is not to find "XMAS" byt rather X-"MAS"; that is, two "MAS" words in an X pattern. The pattern can be any of these four:

     M.S     S.M     M.M     S.S
     .A.     .A.     .A.     .A.
     M.S     S.M     S.S     M.M

I decided to find these by looking for each instance of the middle letter ("A") in the grid, and then, for each pair of diagonal directions, see if the target word ("MAS") can be spelled in both directions:

In [18]:
diagonal_pairs = ([SE, NE], [SW, NW],  [SE, SW], [NE, NW])

def x_search(grid: Grid, word='MAS') -> int:
    """How many times does an X-MAS appear in the grid?"""
    return quantify((grid_can_spell(grid, sub(mid_pos, dir1), dir1, word) and
                     grid_can_spell(grid, sub(mid_pos, dir2), dir2, word))
                    for mid_pos in grid if grid[mid_pos] == word[1]
                    for dir1, dir2 in diagonal_pairs)

answer(4.2, 1822, lambda:
       x_search(xmas_grid))

Puzzle  4.2:   .0605 seconds, answer 1822            ok

# [Day 5](https://adventofcode.com/2024/day/5): Print Queue

Today's puzzle involves a **sleigh launch safety manual** that needs to be updated. The day's input is in two parts: the first a set of **rules** such as "47|53", which means that page 47 must be printed before page 53; and the second a list of **updates** of the form "75,47,61,53,29", meaning that those pages are to be printed in that order.

<img src="https://pbs.twimg.com/media/GeEU0XgWAAARMw-?format=jpg&name=medium" width=400>

I mostly like my `parse` function, but I admit it is not ideal when an input file has two parts like this. I'll parse the two parts into paragraphs, and then call `parse` again on each paragraph:

In [19]:
manual  = parse(5, sections=paragraphs)
rules   = set(parse(manual[0], ints))
updates = parse(manual[1], ints)

assert (48, 39) in rules # `rules` is a set of (earlier, later) page number pairs
assert updates[0] == (61,58,51,32,12,14,71) # `updates` is a sequence of page number tuples

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1366 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
48|39
39|84
39|23
95|51
95|76
95|61
14|52
14|49
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 2 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
48|39
39|84
39|23
95|51
95|76
95|61
14|52
14|49
14|39
14|53
85|19
85|25
85|61
85|35
85|58
74|86
 ...
61,58,51,32,12,14,71
58,25,54,14,12,94,32,76,39
35,53,26,77,14,71,25,76,85,55,51,49,95
32,91,76, ...


### Part 1: What do you get if you add up the middle page number from the correctly-ordered updates?

I'll define `is_correct` to determine if an update is in the correct order, and `sum_of_correct_middles` to add up the middle numbers of the correct updates:

In [20]:
def sum_of_correct_middles(rules: Set[Ints], updates: Tuple[Ints]) -> int:
    """The sum of the middle elements of each update that is correct."""
    return sum(middle(update) for update in updates if is_correct(update, rules))

def is_correct(update: Ints, rules: Set[Ints]) -> bool:
    """An update is correct if no pair of pages violates a rule in the rules set."""
    return not any((second, first) in rules for (first, second) in combinations(update, 2))

def middle(sequence) -> object: return sequence[len(sequence) // 2]

In [21]:
answer(5.1, 5762, lambda:
       sum_of_correct_middles(rules, updates))

Puzzle  5.1:   .0011 seconds, answer 5762            ok

### Part 2: What do you get if you add up the middle page numbers after correctly re-ordering the incorrect updates?

In Part 2 we have to find the incorrect updates, re-order them into a correct order, and again sum the middle page numbers.
Since I have already defined `is_correct`, i could just generate all permutations of each update and find one that `is_correct`. That would work great if the longest update is only 5 pages long, as it is in the example input. But what is the longest update in the actual input?

In [22]:
max(map(len, updates))

23

That's not great. With 23 numbers there are 23! permutations, which is over 25 sextillion. So instead, here's my strategy:

- `sum_of_corrected_middles` will find the incorrect rules, perform a correction on each, and sum the middle numbers.
- `correction` will sort an update, obeying the rules. It used to be that Python's `sort` method allowed a `cmp` keyword to compare two values; there is vestigial support for this with the `functools.cmp_to_key` function. I will **sort** each update so that page *m* comes before page *n* if (*m*, *n*) is in the rules.
- Sorting will be a sextillion times faster than enumerating permutations.

In [23]:
def sum_of_corrected_middles(rules, updates) -> int:
    """The sum of the middle elements of each update that is correct."""
    incorrect = [update for update in updates if not is_correct(update, rules)]
    corrected = [correction(update, rules) for update in incorrect]
    return sum(map(middle, corrected))
    
def correction(update: Ints, rules) -> Ints:
    """Reorder the update to make it correct."""
    def rule_lookup(m, n): return +1 if (m, n) in rules else -1 
    return sorted(update, key=functools.cmp_to_key(rule_lookup))

In [24]:
answer(5.2, 4130, lambda:
       sum_of_corrected_middles(rules, updates))

Puzzle  5.2:   .0016 seconds, answer 4130            ok

I have to say, I'm pleased that this day I got both parts right the first time (and in fact, the same for the previous days). I was worried I might have my `+1` and `-1` backwards in `cmp_to_key`, but so far, everything has gone very smoothly. (However, even if I started solving right at midnight (which I don't), I don't think I would show up on the leaderboard; I have been good at getting a correct answer the first time, but I'm still way slower than the skilled contest programmers.

# [Day 6](https://adventofcode.com/2024/day/6): Guard Gallivant

Today's input is a 2D map of the manufacturing lab, with "`.`" indicating an empty space, "`#`" indicating an obstruction, and "`^`" indicating the position of the security guard.

In [25]:
lab_grid = Grid(parse(6))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 130 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
........#........................................#......#........#.............................. ...
....................................#......#.....#............#.............#..........#........ ...
......................#.......................................................#................. ...
.......#..#..#....#...#...#....#..............#......#.......#...#................#.......#..... ...
......................#....##...#.......#....#.......................................#.......... ...
...#............................#........................................#...................... ...
....................#............#...............#......#.........#...........#................. ...
............................#......#...#................#.........

### Part 1: How many distinct positions will the guard visit before leaving the mapped area?

The guard follows this protocol: If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.

I'll define `follow_path` to output a list of all the positions the guard occupies. I realize the puzzle is only asking for a count of the positions, but the path might be useful for Part 2, or for debugging, so I'll return it. I worried that it is also possible for a path to become a loop, but the problem statement says that can't happen, so I won't test for it.

In [26]:
def follow_path(grid: Grid, guard='^', facing=North) -> List[Point]:
    """A list of all points in the path followed by the guard."""
    path = grid.findall(guard) # A one-element list of positions, e.g. [(3, 4)]
    while True:
        ahead = add2(path[-1], facing)
        if ahead not in grid:
            return path
        elif grid[ahead] == '#':
            facing = make_turn(facing, 'R')
        else:
            path.append(ahead)

answer(6.1, 5329, lambda: len(set(follow_path(lab_grid))))

Puzzle  6.1:   .0043 seconds, answer 5329            ok

I initially had a **bug**; I asked for the length of the path, not the length of the **set** of positions in the path.
                                  
### Part 2: How many different positions could you choose for an obstruction to put the guard in a loop?

The historians would like to place a single obstacle so that the guard *will* get stuck in a loop, rather than exiting the grid. They want to know all possible positions for the obstacle. What do we know about such positions?
- An obstacle  position must be somewhere on the guard's path, otherwise it would have no effect.
- The instructions say it can't be the guard's initial position.
- A loop is when the guard's path returns to the same position with the same facing. This suggests that my Part 1 solution was not completely helpful: to find duplicate positions in the path I would need a set of position/facing pairs, not just positions.
- I can make slightly less work by only storing the corners of the path: the places where the guard turns. 
- The simplest approach for finding obstacle positions is to temporarily place an obstacle on each point on the path, one at a time, and see if it leads to a loop.
- There are 5,329 positions on the path, so the runtime should be about 5,000 times longer than Part 1; on the order of 10 seconds or so. I'll try it, and if it seems too slow, I'll try to think of something better.

In [27]:
def is_loopy_path(grid: Grid, guard_pos, facing=North) -> bool:
    """Does the path followed by the guard form a loop?"""
    path = {(guard_pos, facing)}
    while True:
        ahead = add2(guard_pos, facing)
        if ahead not in grid:
            return False # Walked off the grid; not a loop
        elif grid[ahead] == '#':
            facing = make_turn(facing, 'R')
            if (guard_pos, facing) in path:
                return True
            path.add((guard_pos, facing))
        else:
            guard_pos = ahead
    
def find_loopy_obstacles(grid: Grid) -> Iterable[Point]:
    """All positions in which placing an obstacle would result in a loopy path for the guard."""
    guard_pos = the(grid.findall('^'))
    for pos in set(follow_path(grid)) - {guard_pos}:
        grid[pos] = '#' # Temporarily place an obstacle 
        if is_loopy_path(grid, guard_pos):
            yield pos
        grid[pos] = '.' # Remove the obstacle

In [28]:
answer(6.2, 2162, lambda:
       quantify(find_loopy_obstacles(lab_grid)))

Puzzle  6.2:  5.3153 seconds, answer 2162            ok

That was a bit slow, but I'll take it. I had a **bug** when I was keeping a set of previously visited states to detect loops; the bug went away when I switched to the step-count limit.

# [Day 7](https://adventofcode.com/2024/day/7): Bridge Repair

The narrative for today involves fixing a bridge, and each line of our input represents a calibration equation for the bridge. Unfortunately, some nearby elephants stole all the operators from the equations, so all that is left are the integers:

In [29]:
equations = parse(7, ints)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 850 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
202998336: 686 9 7 62 2 673
19275222: 361 3 7 170 65 5 223
23101: 7 694 916 4 6
2042426: 6 34 2 423 3
40369523: 8 880 91 45 23
46629044796: 990 471 4 4 796
1839056: 3 42 2 4 3 258 703 4 8
26205: 2 9 5 9 9 4 3 7 44 5 8 7
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 850 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(202998336, 686, 9, 7, 62, 2, 673)
(19275222, 361, 3, 7, 170, 65, 5, 223)
(23101, 7, 694, 916, 4, 6)
(2042426, 6, 34, 2, 423, 3)
(40369523, 8, 880, 91, 45, 23)
(46629044796, 990, 471, 4, 4, 796)
(1839056, 3, 42, 2, 4, 3, 258, 703, 4, 8)
(26205, 2, 9, 5, 9, 9, 4, 3, 7, 44, 5, 8, 7)
...


<img src="https://pbs.twimg.com/media/GeOKVYiX0AAwNMy?format=jpg&name=medium" width=400> 

### Part 1: What is the total calibration result of possibly true equations?

Our task is to find operators to balance each equation. The input "`3267: 81 40 27`" can be made into the equation "`3267 = 81 + 40 * 27`", with the understanding that all evaluations are done left-to-right, so this is "`3267 = ((81 + 40) * 27)`". The two allowable operators are addition and multiplication. Our task is to compute the sum of all the equations that can be balanced.

The straightforward approach is to try both operators on every number. If there are *n* numbers in an equation then there will be 2<sup>*n*-2</sup> possible equations; is that going to be a problem?

In [30]:
max(map(len, equations))

13

No problem! With 13 numbers on a line there are 2<sup>11</sup> = 2048 equations; a small number.  I'll define `can_be_calibrated` to keep a set of `results`, updating the set for each new number and each possible operator. Although the instructions were a bit vague, it appears that when they talk about "numbers" in the equations they mean "positive integers". That means that neither addition nor multiplication can cause a number to decrease, so once a result exceeds the target, we'll drop it.

In [31]:
def can_be_calibrated(numbers: ints, operators=(operator.add, operator.mul)) -> bool:
    """Can the tuple of numbers be calibrated as a correct equation using '+' and '*' ?"""
    target, first, *rest = numbers
    results = {first} # A set of all possible results of the partial computation
    for y in rest:
        results = {op(x, y) for x in results if x <= target for op in operators}
    return target in results

In [32]:
answer(7.1, 1985268524462, lambda:
    sum(numbers[0] for numbers in equations if can_be_calibrated(numbers)))

Puzzle  7.1:   .0406 seconds, answer 1985268524462   ok

### Part 2: What is the total calibration result of possibly true equations, allowing concatenation?

In Part 2, we add a third operator: concatentation. The equation "`192: 17 8 14`" can be balanced by concatenated 17 and 8 to get 178, and then adding 14: "`192 = ((17 || 8) + 14)`". With three operators, the equation with 11 operators now has 3<sup>11</sup> = 177,147 possibilities, almost 100 times more than Part 1, so this will take a few seconds:

In [33]:
operators3 = (operator.add, operator.mul, lambda x, y: int(str(x) + str(y)))
    
answer(7.2, 150077710195188, lambda:
       sum(numbers[0] for numbers in equations if can_be_calibrated(numbers, operators3)))

Puzzle  7.2:  2.6449 seconds, answer 150077710195188 ok

# [Day 8](https://adventofcode.com/2024/day/8): Resonant Collinearity

Another grid input, this one a map of antenna locations. Each different non-"." character denotes an antenna of a given frequency.

In [34]:
antennas = Grid(parse(8))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 50 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
..................................................
.................................C................
.e..........7O....................................
.....................................z............
......................t.........C.......k.........
............h................................9....
.............5.7....O.............9C..............
.......5.O................T.......................
...


### Part 1: How many unique locations within the bounds of the map contain an antinode?

An **antinode** occurs at a point that is perfectly in line with two antennas of the same frequency, but only when one of the antennas is twice as far away as the other.

That means that if two antennas are at points *A* and *B*, then the two antinodal points are at 2*A* - *B* and 2*B* - A. If there are three or more antennas with the same frequency then we consider each pair of them in turn. So all we have to do is group the antennas by frequency, compute the antinodes for each pair with the same frequency, and determine which of those antinodal points are on the grid.

In [35]:
def antinodes(antennas: Grid) -> Set[Point]:
    """The set of all antinodal points in the grid.
    (That is, points that are of distance d and 2d from same frequency antennas.)"""
    return union(antinodes2(A, B, antennas)
                 for points in group_all(antennas).values()
                 for A, B in combinations(points, 2))

def antinodes2(A: Point, B: Point, antennas: Grid) -> Set[Point]:
    """The set of antinodal points for two antenna points, A and B."""
    return {P for P in {sub(mul(A, 2), B), sub(mul(B, 2), A)}
              if P in antennas}

def group_all(antennas: Grid) -> Dict[Char, List[Point]]:
    """A dict of {frequency: [all_points_with_an_antenna_of_that_frequency]}."""
    frequencies = defaultdict(list)
    for point, f in antennas.items():
        if f != '.':
            frequencies[f].append(point)
    return frequencies

answer(8.1, 220, 
       lambda: len(antinodes(antennas)))

Puzzle  8.1:   .0006 seconds, answer 220             ok

### Part 2: How many unique locations within the bounds of the map contain an updated antinode?

For Part 2, the updated definition of antinodes means that they can now occur at any point that is exactly on line with two antennas of the same frequency, regardless of distance. So if the two antennas are *A* and *B* then the antinbodal points are of the form *A* + *k*(*A* - *B*) for any scalar value *k* (positive or negative) that makes the resulting point fall within the grid.

In [36]:
def updated_antinodes(antennas: Grid) -> Set[Point]:
    """The set of all updated antinodal points in the grid.
    (That is, points that are on a line with two same frequency antennas.)"""
    return union(updated_antinodes2(A, B, antennas)
                 for points in group_all(antennas).values()
                 for A, B in combinations(points, 2))

def updated_antinodes2(A: Point, B: Point, antennas: Grid) -> Set[Point]:
    """The set of updated antinodal points for two antenna points, A and B."""
    antinodes = set()
    D = sub(A, B)
    for step in (+1, -1):
        for k in count_from(0, step):
            P = add2(A, mul(D, k))
            if P in antennas:
                antinodes.add(P)
            else:
                break
    return antinodes
                

In [37]:
answer(8.2, 813, lambda:
       len(updated_antinodes(antennas)))

Puzzle  8.2:   .0021 seconds, answer 813             ok

I got both of these right the first time (except for some simple typos like mismatched parens and typing `grid` when I meant the grid called `antennas`), but maybe I should have parameterized the first version of `antinodes` to allow for a different `antinodes2` function.

# [Day 9](https://adventofcode.com/2024/day/9): Disk Fragmenter

Today we're confronted with a computer disk that needs to be compressed to gain some contiguous free space. The contents of the disk is represented in the **disk map** format: a string of digits, where the digits alternate between the number of blocks of a file, followed by the number of blocks of free space. We'll parse that as a tuple of digits:

In [38]:
disk_map = the(parse(9, digits))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1 str:
────────────────────────────────────────────────────────────────────────────────────────────────────
692094513253604282899448234539616972499153261626907217394161512944107098953354935354419233821564 ...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 1 tuple:
────────────────────────────────────────────────────────────────────────────────────────────────────
(6, 9, 2, 0, 9, 4, 5, 1, 3, 2, 5, 3, 6, 0, 4, 2, 8, 2, 8, 9, 9, 4, 4, 8, 2, 3, 4, 5, 3, 9, 6, 1, ...


<img src="https://pbs.twimg.com/media/GeYt7iuWIAAHxhT?format=jpg&name=medium" width=400>

### Part 1: Compress the hard drive. What is the resulting filesystem checksum? 

The disk map "`12345`" means that there is 1 block for the first file (which has ID number 0), followed by 2 empty blocks, then 3 blocks for the second file (with ID number 1), followed by 4 empty blocks, and finally 5 blocks for the third file (with ID number 2).  It makes sense to convert this into a **disk layout** format, which would be "`0..111....22222`", where "`.`" represents an empty block.

To **compress** a disk layout, move file blocks one at a time starting by taking the rightmost non-empty block and moving it to the leftmost empty position; repeat until no more moves are possible.

The final answer is a **checksum** of the compressed disk: the sum of the product of the block position times the file ID number for all non-empty blocks.

In [39]:
empty = '.'

def disk_layout(disk_map: Ints) -> list:
    """Convert a disk map into a disk layout."""
    layout = []
    for id, i in enumerate(range(0, len(disk_map), 2)):
        layout.extend(disk_map[i] * [id])
        if i + 1 < len(disk_map):
            layout.extend(disk_map[i + 1] * [empty])
    return layout

def compress_layout(layout: list) -> list:
    """Mutate layout by moving blocks one at a time from the end to the leftmost free space."""
    N    = len(layout)
    free = -1  # Start looking for free space from the left
    end  = N   # Start looking for non-empty blocks from the right
    while True:
        free = first(i for i in range(free + 1, N)    if layout[i] is empty)
        end  = first(i for i in range(end - 1, 0, -1) if layout[i] is not empty)
        if free is None or free >= end:
            return layout
        layout[free], layout[end] = layout[end], empty

def checksum(layout: list) -> list:
    """The sum of the product of the block position times the file ID number for all non-empty blocks."""
    return sum(i * id for i, id in enumerate(layout) if id is not empty)

In [40]:
answer(9.1, 6332189866718, lambda:
       checksum(compress_layout(disk_layout(disk_map))))

Puzzle  9.1:   .0433 seconds, answer 6332189866718   ok

### Part 2: Compress the hard drive with the new method. What is the resulting filesystem checksum? 

In Part 2, there is a new method of compressing the disk, where we move full files rather than a block at a time. Again we start on the right, and try to move a file to the leftmost position where it will fit. If there is no such position, the file doesn't move. `compress_layout2`  implements this new method, performing a move by swapping two [**slices**](https://docs.python.org/3/library/functions.html#slice) of the disk layout: 

    layout[file], layout[free] = layout[free], layout[file]`

To find all the slices that indicate files, it is easier to run through the disk map than the disk layout. The function `file_slices` quickly finds all such slices.

Finding a free space for a file is more difficult, because we need to find one that is big enough. I'll run through the whole layout from left-to-right each time. This will make it *O*(*n*<sup>2</sup>) rather than *O*(*n*), but hopefully it won't be too slow. (If I wanted to speed it up I could have an array of starting positions for each desired size of free space.)

In [41]:
def compress_layout2(disk_map: Ints) -> list:
    """Mutate layout by moving files one at a time from the end to the leftmost free space."""
    layout = disk_layout(disk_map)
    for file in file_slices(disk_map):
        free = find_freespace(layout, file)
        if free:
            layout[file], layout[free] = layout[free], layout[file]
    return layout

def file_slices(disk_map: Ints) -> List[slice]:
    """Given a disk map, find all the slice positions of files in the disk layout (last one first)."""
    slices = []
    block = 0
    for i, length in enumerate(disk_map):
        if i % 2 == 0:
            slices.append(slice(block, block + length))
        block += length
    slices.reverse()
    return slices

def find_freespace(layout, file_slice) -> Optional[slice]:
    """Find a slice position big enough to fit the given file slice, or return None if there is no position."""
    length = file_slice.stop - file_slice.start
    run = 0
    for i in range(layout.index(empty), len(layout)):
        if i >= file_slice.start:
            return None # We only want to move a file left, not right
        elif layout[i] is empty:
            run += 1
            if run == length:
                return slice(i + 1 - length, i + 1)
        else:
            run = 0
    return None

In [42]:
answer(9.2, 6353648390778, lambda:
       checksum(compress_layout2(disk_map)))

Puzzle  9.2:  6.2098 seconds, answer 6353648390778   ok

I got the right answer, but I confess I had an off-by-one **bug** in `find_freespace` on the first try.

# [Day 10](https://adventofcode.com/2024/day/10): Hoof It

Today's input is a topological map, with digits indicating the elevation of each terrain position.

In [43]:
topo = Grid(parse(10, digits))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 60 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
432109865210212123765432101234321098543289654320132112121058
045678774324301012892343023445456787650198763013241001034569
187678789465692321001056014896234986456787012894653212123678
296589921056789433217837895687145675323891233765784589238987
345437835434576544786921278761010014210710321212098676521067
032126546323465435695430789760121223121653450303145125430678
123010567810156543212345699859834321056544067654236012321589
543213498987657665401030787348765430187432198765987622345432
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 60 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(4, 3, 2, 1, 0, 9, 8, 6, 5, 2, 1, 0, 2, 1, 2,

### Part 1: What is the sum of the scores of all trailheads on your topographic map?

A **trailhead** is any position with elevation  0, and a **peak** is any position with elevation 9. The **score** of a trailhead is the number of peaks that can be reached by following a path where each step increases the elevation by exactly 1. All moves are in one of the four cardinal directions (north/south/east/west).

I'll keep a set of points on the frontier of possible paths, updating this set on each iteratation from 1 to 9, by looking at each point on the frontier and seeing which of the neighboring points `p` have the right elevation:

In [44]:
def score(topo: Grid, trailhead: Point) -> int:
    """How many peaks can be reached from this trailhead?"""
    frontier = {trailhead}
    for elevation in range(1, 10):
        frontier = {p for p in union(map(topo.neighbors, frontier))
                    if topo[p] == elevation}
    return len(frontier)

In [45]:
answer(10.1, 744, lambda:
       sum(score(topo, head) for head in topo.findall([0])))

Puzzle 10.1:   .0141 seconds, answer 744             ok

### Part 2: What is the sum of the ratings of all trailheads?

The **rating** of a trailhead is the number of distinct paths from the trailhead to a peak.

As in Part 1, I'll keep a frontier and update it on each iteration from 1 to 9, but this time the frontier will be a counter of `{position: count}` where the count indicates the number of paths to that position. On each iteration I'll look at each point `f` on the frontier and see which of the neighboring points `p` have the right elevation, and increment the counts for those points by the count for `f`. This approach is linear in the number of positions, whereas if I followed all possible paths depth-first there could be an exponential number of paths.

In [46]:
def rating(topo: Grid, trailhead: Point) -> int:
    """How many distinct paths are there from this trailhead to any peak?"""
    frontier = Counter({trailhead: 1})
    for elevation in range(1, 10):
        frontier = accumulate((p, frontier[f]) 
                              for f in frontier
                              for p in topo.neighbors(f) if topo[p] == elevation)
    return sum(frontier.values())

In [47]:
answer(10.2, 1651, lambda:
       sum(rating(topo, head) for head in topo.findall([0])))

Puzzle 10.2:   .0161 seconds, answer 1651            ok

Today I went pretty fast (for me); I started a few minutes late and finished in 15 minutes. From the point of view of a competitive coder I did foolish things like write docstrings and use variables of more than one letter, so while this time was fast for me, it placed well out of the top 1000.

# [Day 11](https://adventofcode.com/2024/day/11): Plutonian Pebbles

Today's input is a single line consisting of a list of integers, representing numbers enscribed on some stones, which are arranged in a straight line.

In [48]:
stones = the(parse(11, ints))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1 str:
────────────────────────────────────────────────────────────────────────────────────────────────────
0 27 5409930 828979 4471 3 68524 170
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 1 tuple:
────────────────────────────────────────────────────────────────────────────────────────────────────
(0, 27, 5409930, 828979, 4471, 3, 68524, 170)


### Part 1: How many stones will you have after blinking 25 times?

Every time you blink, the stones appear to change, according to these rules:
- A stone marked 0 changes to 1.
- Otherwise, a stone with an even number of digits splits into two stones, with the first and second halves of those digits.
- Otherwise, the stone's number is multiplied by 2024.

I'll define `blink` to simulate the effect of a given number of blinks, and `change_stone` to change a single stone, returning a list of wither one or two stones (the two stones computed by `split_stone`):

In [49]:
def blink(stones: Ints, blinks=25) -> List[int]:
    """Simulate the changes in the list of stones after blinking `blinks` times."""
    for _ in range(blinks):
        stones = append(map(change_stone, stones))
    return stones
    
def change_stone(stone: int) -> List[int]:
    """Change a single stone into one or two, according to the rules."""
    digits = str(stone)
    return ([1]                 if stone == 0           else
            split_stone(digits) if len(digits) % 2 == 0 else
            [stone * 2024])

def split_stone(digits: str) -> List[int]:
    """Split a stone into two halves."""
    half = len(digits) // 2
    return [int(digits[:half]), int(digits[half:])]

In [50]:
answer(11.1, 194482, lambda:
       len(blink(stones)))

Puzzle 11.1:   .1570 seconds, answer 194482          ok

### Part 2: How many stones would you have after blinking a total of 75 times?

It looks like the number of stones is roughly doubling every 2 blinks, so for 75 blinks we could have trillions of stones. I'd like something more efficient. I note that:
- Although the puzzle makes it clear that the stones are in a line, it turns out their position in the line is irrelevant.
- Because all the even-digit numbers get split in half, it seems like many small numbers will appear multiple times. In the example, after 6 blinks the number 2 appears 4 times.
- Therefore, I'll keep a `Counter` of stones rather than a list of stones.

In [51]:
def blink2(stones: Ints], blinks=25) -> Counter:
    """Simulate the changes after blinking `blinks` times and return a Counter of stones."""
    counts = Counter(stones)
    for _ in range(blinks):
        counts = accumulate((s, counts[stone]) 
                            for stone in counts 
                            for s in change_stone(stone))
    return counts

Now we can re-run Part 1 (it should be slightly faster), and run Part 2 without fear of having trillion-element lists:

In [52]:
answer(11.1, 194482, lambda:
       total(blink2(stones, 25)))

Puzzle 11.1:   .0037 seconds, answer 194482          ok

In [53]:
answer(11.2, 232454623677743, lambda:
       total(blink2(stones, 75)))

Puzzle 11.2:   .1285 seconds, answer 232454623677743 ok

Again, I did pretty well, with no errors, and moving at what I thought was a good pace, but I didn't even crack the top 2000 on the leaderboard. I guess I spent too much time writing docstrings and type hints, and refactoring as I go.

# Summary

So far, I've solved all the puzzles. Most of them run in well under a second, but three of them take multiple seconds.

In [54]:
for d in sorted(answers):
    print(answers[d])

Puzzle  1.1:   .0002 seconds, answer 1830467         ok
Puzzle  1.2:   .0002 seconds, answer 26674158        ok
Puzzle  2.1:   .0011 seconds, answer 257             ok
Puzzle  2.2:   .0077 seconds, answer 328             ok
Puzzle  3.1:   .0014 seconds, answer 156388521       ok
Puzzle  3.2:   .0009 seconds, answer 75920122        ok
Puzzle  4.1:   .0742 seconds, answer 2401            ok
Puzzle  4.2:   .0605 seconds, answer 1822            ok
Puzzle  5.1:   .0011 seconds, answer 5762            ok
Puzzle  5.2:   .0016 seconds, answer 4130            ok
Puzzle  6.1:   .0043 seconds, answer 5329            ok
Puzzle  6.2:  5.3153 seconds, answer 2162            ok
Puzzle  7.1:   .0406 seconds, answer 1985268524462   ok
Puzzle  7.2:  2.6449 seconds, answer 150077710195188 ok
Puzzle  8.1:   .0006 seconds, answer 220             ok
Puzzle  8.2:   .0021 seconds, answer 813             ok
Puzzle  9.1:   .0433 seconds, answer 6332189866718   ok
Puzzle  9.2:  6.2098 seconds, answer 63536483907